In [ ]:
import spectHR as cs
import pyhrv

In [ ]:
DataSet = cs.SpectHRDataset("Example Data/SUB_002.xdf", 1, event_index = 0) 

In [ ]:
bDataSet = cs.borderData(DataSet)
#bDataSet.ecg = bDataSet.ecg.slicetime(500,2100)

In [ ]:
fDataSet = cs.filterECGData(bDataSet, 
                          {'filterType': 'highpass', 
                           'cutoff': 1})

In [ ]:
tDataSet = cs.calcPeaks(bDataSet)

In [ ]:
%matplotlib widget
GUI = cs.spectplot(tDataSet)

In [ ]:
GUI